In [2]:
from collections import defaultdict

import numpy as np

from rdkit import Chem

import torch

- ref from [Graph neural network (GNN) for molecular property prediction (SMILES format)](https://github.com/masashitsubaki/molecularGNN_smiles)

In [3]:
def create_atoms(mol, atom_dict):
    """Transform the atom types in a molecule (e.g., H, C, and O)
    into the indices (e.g., H=0, C=1, and O=2).
    Note that each atom index considers the aromaticity.
    """
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)


def create_ijbonddict(mol, bond_dict):
    """Create a dictionary, in which each key is a node ID
    and each value is the tuples of its neighboring node
    and chemical bond (e.g., single and double) IDs.
    """
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(radius, atoms, i_jbond_dict,
                         fingerprint_dict, edge_dict):
    """Extract the fingerprints from a molecular graph
    based on Weisfeiler-Lehman algorithm.
    """

    if (len(atoms) == 1) or (radius == 0):
        nodes = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges.
            The updated node IDs are the fingerprint IDs.
            """
            nodes_ = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                nodes_.append(fingerprint_dict[fingerprint])

            """Also update each edge ID considering
            its two nodes on both sides.
            """
            i_jedge_dict_ = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    i_jedge_dict_[i].append((j, edge))

            nodes = nodes_
            i_jedge_dict = i_jedge_dict_

    return np.array(nodes)


def split_dataset(dataset, ratio):
    """Shuffle and split a dataset."""
    np.random.seed(1234)  # fix the seed for shuffle.
    np.random.shuffle(dataset)
    n = int(ratio * len(dataset))
    return dataset[:n], dataset[n:]


def create_datasets(task, dataset, radius, device):

    dir_dataset = '../dataset/' + task + '/' + dataset + '/'

    """Initialize x_dict, in which each key is a symbol type
    (e.g., atom and chemical bond) and each value is its index.
    """
    atom_dict = defaultdict(lambda: len(atom_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))

    def create_dataset(filename):

        print(filename)

        """Load a dataset."""
        with open(dir_dataset + filename, 'r') as f:
            smiles_property = f.readline().strip().split()
            data_original = f.read().strip().split('\n')

        """Exclude the data contains '.' in its smiles."""
        data_original = [data for data in data_original
                         if '.' not in data.split()[0]]

        dataset = []

        for data in data_original:

            smiles, property = data.strip().split()

            """Create each data with the above defined functions."""
            mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
            atoms = create_atoms(mol, atom_dict)
            molecular_size = len(atoms)
            i_jbond_dict = create_ijbonddict(mol, bond_dict)
            fingerprints = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                fingerprint_dict, edge_dict)
            adjacency = Chem.GetAdjacencyMatrix(mol)

            """Transform the above each data of numpy
            to pytorch tensor on a device (i.e., CPU or GPU).
            """
            fingerprints = torch.LongTensor(fingerprints).to(device)
            adjacency = torch.FloatTensor(adjacency).to(device)
            if task == 'classification':
                property = torch.LongTensor([int(property)]).to(device)
            if task == 'regression':
                property = torch.FloatTensor([[float(property)]]).to(device)

            dataset.append((fingerprints, adjacency, molecular_size, property))

        return dataset

    dataset_train = create_dataset('data_train.txt')
    dataset_train, dataset_dev = split_dataset(dataset_train, 0.9)
    dataset_test = create_dataset('data_test.txt')

    N_fingerprints = len(fingerprint_dict)

    return dataset_train, dataset_dev, dataset_test, N_fingerprints

In [10]:
atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
edge_dict = defaultdict(lambda: len(edge_dict))

In [19]:
radius=1
dim=50
layer_hidden=6
layer_output=6

batch_train=32
batch_test=32
lr=1e-4
lr_decay=0.99
decay_interval=10
iteration=1000

task='classification'

- [PyTorch can't find the name?? (NameError: name 'device' is not defined)](https://stackoverflow.com/questions/66679163/pytorch-cant-find-the-name-nameerror-name-device-is-not-defined)

In [20]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
dataset = []

for data in data_original:

    smiles, property = data.strip().split()

    """Create each data with the above defined functions."""
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
#     print ('mol', mol)
    atoms = create_atoms(mol, atom_dict)
    print ('atoms', atoms)
    molecular_size = len(atoms)
    print ('molecular_size', molecular_size)
    i_jbond_dict = create_ijbonddict(mol, bond_dict)
    print ('i_jbond_dict', i_jbond_dict)
    fingerprints = extract_fingerprints(radius, atoms, i_jbond_dict,
                                        fingerprint_dict, edge_dict)
    print ('fingerprints', fingerprints)
    adjacency = Chem.GetAdjacencyMatrix(mol)
    print ('adjacency', adjacency)

    """Transform the above each data of numpy
    to pytorch tensor on a device (i.e., CPU or GPU).
    """
    fingerprints = torch.LongTensor(fingerprints).to(device)
    adjacency = torch.FloatTensor(adjacency).to(device)
    if task == 'classification':
        property = torch.LongTensor([int(property)]).to(device)
    if task == 'regression':
        property = torch.FloatTensor([[float(property)]]).to(device)

    dataset.append((fingerprints, adjacency, molecular_size, property))

atoms [0 0 1 2 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 5 5 5 5 5 5 5 5 5 5 5 5 5]
molecular_size 34
i_jbond_dict defaultdict(<function create_ijbonddict.<locals>.<lambda> at 0x11bc21ee0>, {0: [(1, 0), (21, 0), (22, 0), (23, 0)], 1: [(0, 0), (2, 1), (3, 0)], 2: [(1, 1)], 3: [(1, 0), (4, 0), (12, 0)], 4: [(3, 0), (5, 2), (9, 2)], 5: [(4, 2), (6, 2), (24, 0)], 6: [(5, 2), (7, 2), (25, 0)], 7: [(6, 2), (8, 2), (26, 0)], 8: [(7, 2), (9, 2), (27, 0)], 9: [(8, 2), (10, 0), (4, 2)], 10: [(9, 0), (11, 0)], 11: [(10, 0), (12, 2), (20, 2)], 12: [(11, 2), (13, 2), (3, 0)], 13: [(12, 2), (14, 2), (28, 0)], 14: [(13, 2), (15, 2), (29, 0)], 15: [(14, 2), (16, 2), (20, 2)], 16: [(15, 2), (17, 2), (30, 0)], 17: [(16, 2), (18, 2), (31, 0)], 18: [(17, 2), (19, 2), (32, 0)], 19: [(18, 2), (20, 2), (33, 0)], 20: [(19, 2), (15, 2), (11, 2)], 21: [(0, 0)], 22: [(0, 0)], 23: [(0, 0)], 24: [(5, 0)], 25: [(6, 0)], 26: [(7, 0)], 27: [(8, 0)], 28: [(13, 0)], 29: [(14, 0)], 30: [(16, 0)], 31: [(17, 0)], 32: [(18, 0)], 3

In [36]:
dataset

[(tensor([ 0,  1,  2,  3,  4,  5,  5,  5,  5,  6,  7,  6,  4,  5,  5,  8,  5,  5,
           5,  5,  8,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]),
  tensor([[0., 1., 0.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  34,
  tensor([0])),
 (tensor([11,  4,  5,  5, 12, 13, 13, 12,  5,  5,  4, 11,  5,  6, 14, 15, 15, 16,
           6, 14, 15, 15, 16,  5, 17, 17, 10, 10,  9,  9, 10, 10, 17, 17, 10, 18,
          18, 10]),
  tensor([[0., 1., 0.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  38,
  tensor([0])),
 (tensor([ 0, 19, 20, 21, 22, 23,  4,  5,  5,  5, 24, 25,  5, 20, 19,  0,  9

In [26]:
N = len(dataset)
batch_train= 8

for i in range(0, N, batch_train):
    data_batch = list(zip(*dataset[i:i+batch_train]))

In [27]:
data_batch

[(tensor([ 11,  88,  70, 102,  45,  71,  70, 103, 104,  87,  70,  17,  17,  46,
           10,  99]),
  tensor([ 45, 102,  70, 103,  70,  87, 104, 103,  70,  71,  46,  10,  99,  10]),
  tensor([105, 106,  28,   5,  12, 107,  28, 106, 105,  28, 106, 105, 107,   5,
            5,  28, 106, 105, 108, 109,   5,  12, 110,  77,   1,   0,   2,  43,
           56,   9,   9,   9,  10,   9,   9,   9,   9,   9,   9,  10,  10,   9,
            9,   9,  10,   9,  17,   9,   9,   9,   9,   9,   9,   9]),
  tensor([ 79,  80,  66, 111, 112,  66, 111, 112,   9,   9,   9,   9,   9,   9,
            9,   9,   9,   9,   9]),
  tensor([100, 113, 100,  15,   9,   9,   9,   9,   9,   9]),
  tensor([  0,  43,  43,  19, 114, 115, 114,  19,  43,  43,   0, 114,  19,  43,
           43,   0,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
            9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
            9])),
 (tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 

In [28]:
inputs = data_batch[:-1]
correct_labels = torch.cat(data_batch[-1])

In [29]:
inputs

[(tensor([ 11,  88,  70, 102,  45,  71,  70, 103, 104,  87,  70,  17,  17,  46,
           10,  99]),
  tensor([ 45, 102,  70, 103,  70,  87, 104, 103,  70,  71,  46,  10,  99,  10]),
  tensor([105, 106,  28,   5,  12, 107,  28, 106, 105,  28, 106, 105, 107,   5,
            5,  28, 106, 105, 108, 109,   5,  12, 110,  77,   1,   0,   2,  43,
           56,   9,   9,   9,  10,   9,   9,   9,   9,   9,   9,  10,  10,   9,
            9,   9,  10,   9,  17,   9,   9,   9,   9,   9,   9,   9]),
  tensor([ 79,  80,  66, 111, 112,  66, 111, 112,   9,   9,   9,   9,   9,   9,
            9,   9,   9,   9,   9]),
  tensor([100, 113, 100,  15,   9,   9,   9,   9,   9,   9]),
  tensor([  0,  43,  43,  19, 114, 115, 114,  19,  43,  43,   0, 114,  19,  43,
           43,   0,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
            9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
            9])),
 (tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 

In [30]:
correct_labels

tensor([0, 0, 0, 0, 0, 0])

In [47]:
print (type(inputs))
print (len(inputs))
for i in range(len(inputs)):
    print (type(inputs[i]))

<class 'list'>
3
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [39]:
fingerprints, adjacencies, molecular_sizes = inputs
# fingerprints = torch.cat(fingerprints)

In [40]:
fingerprints

(tensor([ 11,  88,  70, 102,  45,  71,  70, 103, 104,  87,  70,  17,  17,  46,
          10,  99]),
 tensor([ 45, 102,  70, 103,  70,  87, 104, 103,  70,  71,  46,  10,  99,  10]),
 tensor([105, 106,  28,   5,  12, 107,  28, 106, 105,  28, 106, 105, 107,   5,
           5,  28, 106, 105, 108, 109,   5,  12, 110,  77,   1,   0,   2,  43,
          56,   9,   9,   9,  10,   9,   9,   9,   9,   9,   9,  10,  10,   9,
           9,   9,  10,   9,  17,   9,   9,   9,   9,   9,   9,   9]),
 tensor([ 79,  80,  66, 111, 112,  66, 111, 112,   9,   9,   9,   9,   9,   9,
           9,   9,   9,   9,   9]),
 tensor([100, 113, 100,  15,   9,   9,   9,   9,   9,   9]),
 tensor([  0,  43,  43,  19, 114, 115, 114,  19,  43,  43,   0, 114,  19,  43,
          43,   0,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
           9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
           9]))

In [48]:
adjacencies

(tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0.,

In [49]:
molecular_sizes

(16, 14, 54, 19, 10, 43)

In [42]:
fingerprints = torch.cat(fingerprints)
fingerprints

tensor([ 11,  88,  70, 102,  45,  71,  70, 103, 104,  87,  70,  17,  17,  46,
         10,  99,  45, 102,  70, 103,  70,  87, 104, 103,  70,  71,  46,  10,
         99,  10, 105, 106,  28,   5,  12, 107,  28, 106, 105,  28, 106, 105,
        107,   5,   5,  28, 106, 105, 108, 109,   5,  12, 110,  77,   1,   0,
          2,  43,  56,   9,   9,   9,  10,   9,   9,   9,   9,   9,   9,  10,
         10,   9,   9,   9,  10,   9,  17,   9,   9,   9,   9,   9,   9,   9,
         79,  80,  66, 111, 112,  66, 111, 112,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   9,   9, 100, 113, 100,  15,   9,   9,   9,   9,   9,
          9,   0,  43,  43,  19, 114, 115, 114,  19,  43,  43,   0, 114,  19,
         43,  43,   0,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9])

In [ ]:
import sys
import timeit

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import roc_auc_score

import preprocess as pp


class MolecularGraphNeuralNetwork(nn.Module):
    def __init__(self, N_fingerprints, dim, layer_hidden, layer_output):
        super(MolecularGraphNeuralNetwork, self).__init__()
        self.embed_fingerprint = nn.Embedding(N_fingerprints, dim)
        self.W_fingerprint = nn.ModuleList([nn.Linear(dim, dim)
                                            for _ in range(layer_hidden)])
        self.W_output = nn.ModuleList([nn.Linear(dim, dim)
                                       for _ in range(layer_output)])
        if task == 'classification':
            self.W_property = nn.Linear(dim, 2)
        if task == 'regression':
            self.W_property = nn.Linear(dim, 1)

    def pad(self, matrices, pad_value):
        """Pad the list of matrices
        with a pad_value (e.g., 0) for batch processing.
        For example, given a list of matrices [A, B, C],
        we obtain a new matrix [A00, 0B0, 00C],
        where 0 is the zero (i.e., pad value) matrix.
        """
        shapes = [m.shape for m in matrices]
        M, N = sum([s[0] for s in shapes]), sum([s[1] for s in shapes])
        zeros = torch.FloatTensor(np.zeros((M, N))).to(device)
        pad_matrices = pad_value + zeros
        i, j = 0, 0
        for k, matrix in enumerate(matrices):
            m, n = shapes[k]
            pad_matrices[i:i+m, j:j+n] = matrix
            i += m
            j += n
        return pad_matrices

    def update(self, matrix, vectors, layer):
        hidden_vectors = torch.relu(self.W_fingerprint[layer](vectors))
        return hidden_vectors + torch.matmul(matrix, hidden_vectors)

    def sum(self, vectors, axis):
        sum_vectors = [torch.sum(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(sum_vectors)

    def mean(self, vectors, axis):
        mean_vectors = [torch.mean(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(mean_vectors)

    def gnn(self, inputs):

        """Cat or pad each input data for batch processing."""
        fingerprints, adjacencies, molecular_sizes = inputs
        fingerprints = torch.cat(fingerprints)
        adjacencies = self.pad(adjacencies, 0)

        """GNN layer (update the fingerprint vectors)."""
        fingerprint_vectors = self.embed_fingerprint(fingerprints)
        for l in range(layer_hidden):
            hs = self.update(adjacencies, fingerprint_vectors, l)
            fingerprint_vectors = F.normalize(hs, 2, 1)  # normalize.

        """Molecular vector by sum or mean of the fingerprint vectors."""
        molecular_vectors = self.sum(fingerprint_vectors, molecular_sizes)
        # molecular_vectors = self.mean(fingerprint_vectors, molecular_sizes)

        return molecular_vectors

    def mlp(self, vectors):
        """Classifier or regressor based on multilayer perceptron."""
        for l in range(layer_output):
            vectors = torch.relu(self.W_output[l](vectors))
        outputs = self.W_property(vectors)
        return outputs

    def forward_classifier(self, data_batch, train):

        inputs = data_batch[:-1]
        correct_labels = torch.cat(data_batch[-1])

        if train:
            molecular_vectors = self.gnn(inputs)
            predicted_scores = self.mlp(molecular_vectors)
            loss = F.cross_entropy(predicted_scores, correct_labels)
            return loss
        else:
            with torch.no_grad():
                molecular_vectors = self.gnn(inputs)
                predicted_scores = self.mlp(molecular_vectors)
            predicted_scores = predicted_scores.to('cpu').data.numpy()
            predicted_scores = [s[1] for s in predicted_scores]
            correct_labels = correct_labels.to('cpu').data.numpy()
            return predicted_scores, correct_labels

    def forward_regressor(self, data_batch, train):

        inputs = data_batch[:-1]
        correct_values = torch.cat(data_batch[-1])

        if train:
            molecular_vectors = self.gnn(inputs)
            predicted_values = self.mlp(molecular_vectors)
            loss = F.mse_loss(predicted_values, correct_values)
            return loss
        else:
            with torch.no_grad():
                molecular_vectors = self.gnn(inputs)
                predicted_values = self.mlp(molecular_vectors)
            predicted_values = predicted_values.to('cpu').data.numpy()
            correct_values = correct_values.to('cpu').data.numpy()
            predicted_values = np.concatenate(predicted_values)
            correct_values = np.concatenate(correct_values)
            return predicted_values, correct_values


class Trainer(object):
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def train(self, dataset):
        np.random.shuffle(dataset)
        N = len(dataset)
        loss_total = 0
        for i in range(0, N, batch_train):
            data_batch = list(zip(*dataset[i:i+batch_train]))
            if task == 'classification':
                loss = self.model.forward_classifier(data_batch, train=True)
            if task == 'regression':
                loss = self.model.forward_regressor(data_batch, train=True)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.item()
        return loss_total


class Tester(object):
    def __init__(self, model):
        self.model = model

    def test_classifier(self, dataset):
        N = len(dataset)
        P, C = [], []
        for i in range(0, N, batch_test):
            data_batch = list(zip(*dataset[i:i+batch_test]))
            predicted_scores, correct_labels = self.model.forward_classifier(
                                               data_batch, train=False)
            P.append(predicted_scores)
            C.append(correct_labels)
        AUC = roc_auc_score(np.concatenate(C), np.concatenate(P))
        return AUC

    def test_regressor(self, dataset):
        N = len(dataset)
        SAE = 0  # sum absolute error.
        for i in range(0, N, batch_test):
            data_batch = list(zip(*dataset[i:i+batch_test]))
            predicted_values, correct_values = self.model.forward_regressor(
                                               data_batch, train=False)
            SAE += sum(np.abs(predicted_values-correct_values))
        MAE = SAE / N  # mean absolute error.
        return MAE

    def save_result(self, result, filename):
        with open(filename, 'a') as f:
            f.write(result + '\n')


if __name__ == "__main__":

    (task, dataset, radius, dim, layer_hidden, layer_output,
     batch_train, batch_test, lr, lr_decay, decay_interval, iteration,
     setting) = sys.argv[1:]
    (radius, dim, layer_hidden, layer_output,
     batch_train, batch_test, decay_interval,
     iteration) = map(int, [radius, dim, layer_hidden, layer_output,
                            batch_train, batch_test,
                            decay_interval, iteration])
    lr, lr_decay = map(float, [lr, lr_decay])

    if torch.cuda.is_available():
        device = torch.device('cuda')
        print('The code uses a GPU!')
    else:
        device = torch.device('cpu')
        print('The code uses a CPU...')
    print('-'*100)

    print('Preprocessing the', dataset, 'dataset.')
    print('Just a moment......')
    (dataset_train, dataset_dev, dataset_test,
     N_fingerprints) = pp.create_datasets(task, dataset, radius, device)
    print('-'*100)

    print('The preprocess has finished!')
    print('# of training data samples:', len(dataset_train))
    print('# of development data samples:', len(dataset_dev))
    print('# of test data samples:', len(dataset_test))
    print('-'*100)

    print('Creating a model.')
    torch.manual_seed(1234)
    model = MolecularGraphNeuralNetwork(
            N_fingerprints, dim, layer_hidden, layer_output).to(device)
    trainer = Trainer(model)
    tester = Tester(model)
    print('# of model parameters:',
          sum([np.prod(p.size()) for p in model.parameters()]))
    print('-'*100)

    file_result = '../output/result--' + setting + '.txt'
    if task == 'classification':
        result = 'Epoch\tTime(sec)\tLoss_train\tAUC_dev\tAUC_test'
    if task == 'regression':
        result = 'Epoch\tTime(sec)\tLoss_train\tMAE_dev\tMAE_test'

    with open(file_result, 'w') as f:
        f.write(result + '\n')

    print('Start training.')
    print('The result is saved in the output directory every epoch!')

    np.random.seed(1234)

    start = timeit.default_timer()

    for epoch in range(iteration):

        epoch += 1
        if epoch % decay_interval == 0:
            trainer.optimizer.param_groups[0]['lr'] *= lr_decay

        loss_train = trainer.train(dataset_train)

        if task == 'classification':
            prediction_dev = tester.test_classifier(dataset_dev)
            prediction_test = tester.test_classifier(dataset_test)
        if task == 'regression':
            prediction_dev = tester.test_regressor(dataset_dev)
            prediction_test = tester.test_regressor(dataset_test)

        time = timeit.default_timer() - start

        if epoch == 1:
            minutes = time * iteration / 60
            hours = int(minutes / 60)
            minutes = int(minutes - 60 * hours)
            print('The training will finish in about',
                  hours, 'hours', minutes, 'minutes.')
            print('-'*100)
            print(result)

        result = '\t'.join(map(str, [epoch, time, loss_train,
                                     prediction_dev, prediction_test]))
        tester.save_result(result, file_result)

        print(result)